In [ ]:
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
from uuid import UUID
from pprint import pprint

### Connect

In [ ]:
username = "analyst"
password = "Esri.4.GIS"
url = "https://ebase.ad.local/server/rest/services/Hosted/SupplyChain/KnowledgeGraphServer"
gis = GIS(r"https://ebase.ad.local/portal", username, password)
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

### Get list of distributor names to process

In [ ]:
distributorNameQuery = """MATCH (p:ProcessingPlant)
WITH DISTINCT p.Distributors as d
RETURN d"""
x=knowledge_graph.query(distributorNameQuery)
y = []
distributorNames = []
for i in range(len(x)):
    y.extend(x[i])
for item in y:
    temp = item.split(", ")
    for s in temp:
        distributorNames.append(s)
distributorNames = list(set(distributorNames))
print(distributorNames)

### Query

In [ ]:
def submitQuery(distributorName):
    opQuery = """MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "{0}")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "Distributor 2"
    RETURN p, d""".format(distributorName)
    print(opQuery)

    queryResults = knowledge_graph.query(opQuery)
    return queryResults

### Given returned nodes, create relationships

In [ ]:
def createRelationships(queryResults):
    countRels = 0
    for p, d in queryResults:
        #print("Related nodes:")
        #print(p['_properties']['PlantName'])
        #print(p['_id'])
        #print(p['_properties']['Material'])
        #print(d['_properties']['DistributorName'])
        #print(d['_id'])
        #print(d['_properties']['Material'])
        #print()
        add_relationship = {
            "_objectType": "relationship",
            "_typeName": "SellsTo",
            "_originEntityId": UUID(f"{d['_id']}"), # Distributor ID
            "_destinationEntityId": UUID(f"{p['_id']}") # Plant ID
        }
        knowledge_graph.apply_edits(adds=[add_relationship])
        countRels += 1
        #print("EDITS MADE")
    print(f"All done! {countRels} relationships created.")

### Perform the edits

In [ ]:
for distributor in distributorNames:
    print(f"Finding related Processing Plants for {distributor}")
    queryNodes = submitQuery(distributor)
    createRelationships(queryNodes) 
print("End of script.")